In [1]:
import tensorflow.keras as keras

In [2]:
keras.__version__

'2.2.4-tf'

In [3]:
epochs = 20
batch_size = 256
learning_rate = 0.001

# for gmf
latent_dim = 8

# for mlp
embedding_size = 64

# regs = [0, 0]

# 모델 확인용.. data를 실제로 넣어서 학습할때는 없애야함.
user_num = 100
item_num = 100

In [4]:
# gmf
# user, item을 동일한 size의 latent space로.. embedding..후 element wise product..
user_input_gmf = keras.layers.Input(shape = (1,), dtype = "int32", name = "user_input_gmf")
item_input_gmf = keras.layers.Input(shape = (1,), dtype = "int32", name = "item_input_gmf")


user_embedding_gmf = keras.layers.Embedding(input_dim = user_num,
                                            output_dim = latent_dim,
                                            name = "user_embedding_gmf",
#                                             embeddings_initializer = "uniform",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

item_embedding_gmf = keras.layers.Embedding(input_dim = item_num,
                                            output_dim = latent_dim,
                                            name = "item_embedding_gmf",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

user_latent_gmf = keras.layers.Flatten()(user_embedding_gmf(user_input_gmf))
item_latent_gmf = keras.layers.Flatten()(item_embedding_gmf(item_input_gmf))

element_wise_product = keras.layers.multiply([user_latent_gmf, item_latent_gmf])

prediction_gmf = keras.layers.Dense(1,
                                    activation = "sigmoid",
                                    kernel_initializer = "lecun_uniform",
                                    name = "prediction")(element_wise_product)

model_gmf = keras.models.Model(inputs = [user_input_gmf, item_input_gmf],
                               outputs = prediction_gmf)

In [5]:
model_gmf.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input_gmf (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input_gmf (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding_gmf (Embedding)  (None, 1, 8)         800         user_input_gmf[0][0]             
__________________________________________________________________________________________________
item_embedding_gmf (Embedding)  (None, 1, 8)         800         item_input_gmf[0][0]             
______________________________________________________________________________________________

In [15]:
# mlp
# user, item을 임베딩한후..concat, mlp layer 통과후 prediction 추출

user_input_mlp = keras.layers.Input(shape = (1,), dtype = "int32", name = "user_input_mlp")
item_input_mlp = keras.layers.Input(shape = (1,), dtype = "int32", name = "item_input_mlp")

user_embedding_mlp = keras.layers.Embedding(input_dim = user_num,
                                            output_dim = int(embedding_size / 2),
                                            name = "user_embedding_mlp",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

item_embedding_mlp = keras.layers.Embedding(input_dim = item_num,
                                            output_dim = int(embedding_size / 2),
                                            name = "item_embedding_mlp",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

user_latent_mlp = keras.layers.Flatten()(user_embedding_mlp(user_input_mlp)) # 32
item_latent_mlp = keras.layers.Flatten()(item_embedding_mlp(item_input_mlp)) # 32

concat = keras.layers.concatenate([user_latent_mlp, item_latent_mlp]) # 64

mlp_1 = keras.layers.Dense(units = embedding_size / 2, activation = "relu", name = "mlp_1")(concat)
mlp_2 = keras.layers.Dense(units = embedding_size / 4, activation = "relu", name = "mlp_2")(mlp_1)
mlp_3 = keras.layers.Dense(units = embedding_size / 8, activation = "relu", name = "mlp_3")(mlp_2)

prediction_mlp = keras.layers.Dense(1,
                                    activation = "sigmoid",
                                    kernel_initializer = "lecun_uniform",
                                    name = "prediction_mlp")(mlp_3)

model_mlp = keras.models.Model(inputs = [user_input_mlp, item_input_mlp],
                               outputs = prediction_mlp)

In [16]:
model_mlp.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input_mlp (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input_mlp (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding_mlp (Embedding)  (None, 1, 32)        3200        user_input_mlp[0][0]             
__________________________________________________________________________________________________
item_embedding_mlp (Embedding)  (None, 1, 32)        3200        item_input_mlp[0][0]             
____________________________________________________________________________________________

In [ ]:
# data를 로드해서 학습시키고.. pretrain모델을 활용 NeuMF layer를 거쳐 최종 score를 낸다.